# Get Potential Components

In [ ]:
include("src/GetPotentialComponents.jl")
using .GetPotentialComponents

using Plots
using DelimitedFiles

Firstly, you should setup configuration of symmetry functions that was used during training process

`G2(eta, rcutoff, rshift)`

In [ ]:
const G2_FUNCTIONS_LIST = [G2(0.125, 7.0, 0.00),
                           G2(4.000, 7.0, 3.00),
                           G2(4.000, 7.0, 3.50),
                           G2(4.000, 7.0, 4.00),
                           G2(4.000, 7.0, 4.50),
                           G2(4.000, 7.0, 5.00),
                           G2(4.000, 7.0, 5.50),
                           G2(4.000, 7.0, 6.00),
                           G2(4.000, 7.0, 6.50)]

Now you need to set up other parameters:

- `component_type`: it could be `pair`, `3body`, `diff`. 
- `model_file`: here you should specify the path to the neural network file in BSON format.
- `output_file`: this is the name of the file where the results of the calculation will be saved.
- `r_max`: this is the size of the frame in Å. The scanning particle at the start of scanning will be placed in the position with coordinates $(X, Y) = (-r_{max}, r_{max})$ (it's the top left corner). The final position of the scanning will be the symmetrical bottom right corner.
- `step_size`: this is the frequency of calculation, determining how often we calculate values. The smaller the `step_size` , the better the pictures, but the slower the calculation.
- `separation`: this is the distance between two particles in the system with 3 particles (two main and one scanning).

In [ ]:
component_type = "pair"  # pair / 3body / diff
model_file = "example-methanol-model.bson"
output_file = "$(component_type)-component-results.txt"
r_max = 8.0  # in Å
step_size = 0.05  # in Å
separation = 2.5;  # in Å ONLY for 3body

## Pair Component

In [ ]:
component_type = "pair"  # pair / 3body / diff
output_file = "$(component_type)-component-results.txt"
GetPotentialComponents.main(component_type, model_file, output_file, r_max, step_size, G2_FUNCTIONS_LIST, separation)

In [ ]:
filename = "pair-component-results.txt"  # Change for you file
data = readdlm(filename, Float64, skipstart=2)
r = data[:, 1]
U = data[:, 2];

In [ ]:
p = plot(r, U,
    xlabel="r [Å]", ylabel="U(r)",
    title="Pair potential component",
    ylim=(-0.5, 5),  # change for your scale
    legend=false,
    size=(800, 600))

savefig(p, "pair-potential-component-plot.png")
display(p)

## 3body potential

In [ ]:
component_type = "3body"  # pair / 3body / diff
output_file = "$(component_type)-component-results.txt"
GetPotentialComponents.main(component_type, model_file, output_file, r_max, step_size, G2_FUNCTIONS_LIST, separation)

In [ ]:
filename = "3body-component-results.txt"  # Change for you file
data = readdlm(filename, ',', skipstart = 3)

X = data[:, 1]
Y = data[:, 2]
U = data[:, 3]

unique_X = sort(unique(X))
unique_Y = sort(unique(Y))

heatmap_data = zeros(length(unique_Y), length(unique_X))

for i in 1:length(X)
    x_idx = findfirst(isequal(X[i]), unique_X)
    y_idx = findfirst(isequal(Y[i]), unique_Y)
    heatmap_data[y_idx, x_idx] = U[i]
end

In [ ]:
p = heatmap(unique_X, unique_Y,
    heatmap_data,
    xlabel = "X", ylabel = "Y", colorbar_title = "U(r)",
    title = "U(r)",
    color = :Reds,
    aspect_ratio = 1, size = (800, 600),
    ylim = (minimum(unique_Y), maximum(unique_Y)),
    xlim = (minimum(unique_X), maximum(unique_X)))

savefig(p, "3body-potential-plot.png")
display(p)

## 3body component

In [ ]:
component_type = "diff"  # pair / 3body / diff
output_file = "$(component_type)-component-results.txt"
GetPotentialComponents.main(component_type, model_file, output_file, r_max, step_size, G2_FUNCTIONS_LIST, separation)

In [ ]:
filename = "diff-component-results.txt"  # Change for you file
data = readdlm(filename, ',', skipstart = 3)

X = data[:, 1]
Y = data[:, 2]
U = data[:, 3]

unique_X = sort(unique(X))
unique_Y = sort(unique(Y))

heatmap_data = zeros(length(unique_Y), length(unique_X))

for i in 1:length(X)
    x_idx = findfirst(isequal(X[i]), unique_X)
    y_idx = findfirst(isequal(Y[i]), unique_Y)
    heatmap_data[y_idx, x_idx] = U[i]
end


In [ ]:
p = heatmap(unique_X, unique_Y,
    heatmap_data,
    xlabel = "X", ylabel = "Y", colorbar_title = "U(r)",
    title = "U(r)",
    color = :Reds,
    aspect_ratio = 1, size = (800, 600),
    ylim = (minimum(unique_Y), maximum(unique_Y)),
    xlim = (minimum(unique_X), maximum(unique_X)))

savefig(p, "diff-component-plot.png")
display(p)